In [1]:
!pip install pylidc
!pip install pydicom
!pip install opencv-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 17.2 MB/s eta 0:00:0000:0100:01


In [2]:
import numpy as np
import pylidc as pl
import pydicom
import cv2
from statistics import median_high
import os
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D, Dense, BatchNormalization, Activation, AveragePooling2D, concatenate
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import plot_model
from keras.callbacks import ModelCheckpoint, EarlyStopping

import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix, classification_report, recall_score, f1_score


In [3]:
import os
import pydicom
import pylidc as pl
import numpy as np
import cv2
from pylidc.Contour import Contour
from statistics import median_high

# Paths to the directories containing DICOM files
base_dirs = [
    "/kaggle/input/lidcidri30/LIDC-IDRI-0001-0200",
    "/kaggle/input/lidcidri30/LIDC-IDRI-0201-0400",
    "/kaggle/input/lidcidri30/LIDC-IDRI-0401-0600"
]

images = []
labels = []
malignant_count = 0
benign_count = 0
max_malignant_count = 4500  # Target number of malignant images
max_benign_count = 2500     # Target number of benign images

# Patch the Contour class method to_matrix
def patched_to_matrix(self, include_k=True):
    coords = np.array([list(map(int, coord.split(','))) for coord in self.coords.split('\n')])
    ij = np.c_[coords[:, 0], coords[:, 1]]

    if not include_k:
        return ij.astype(np.int64)

    k = np.ones(ij.shape[0]) * self.image_z_position
    return np.c_[ij, k].astype(np.int64)

Contour.to_matrix = patched_to_matrix

def calculate_malignancy(nodule):
    list_of_malignancy = [annotation.malignancy for annotation in nodule]
    malignancy = median_high(list_of_malignancy)
    return 'malignant' if malignancy > 3 else 'benign'

# Process DICOM files with the patched Contour class
for base_dir in base_dirs:
    for dirpath, _, filenames in os.walk(base_dir):
        for filename in filenames:
            if filename.endswith(".dcm"):
                try:
                    dicom_file = os.path.join(dirpath, filename)
                    dicom_data = pydicom.dcmread(dicom_file)
                    pid = dicom_data.PatientID
                    scan = pl.query(pl.Scan).filter(pl.Scan.patient_id == pid).first()

                    if scan is None:
                        continue

                    nodules_annotation = scan.cluster_annotations()

                    try:
                        if len(nodules_annotation) > 0:
                            contains_malignant = False

                            for nodule in nodules_annotation:
                                malignancy = calculate_malignancy(nodule)
                                if malignancy == 'malignant':
                                    contains_malignant = True
                                    break

                            if contains_malignant:
                                if malignant_count >= max_malignant_count:
                                    continue
                                malignancy = 'malignant'
                                malignant_count += 1
                            else:
                                if benign_count >= max_benign_count:
                                    continue
                                malignancy = 'benign'
                                benign_count += 1

                            img = dicom_data.pixel_array  # Assuming DICOM contains pixel data directly
                            img = cv2.resize(img, (300, 300))  # Resize to 300x300 for DensNet201
                            img = img.astype(np.float32)

                            images.append(img)
                            labels.append(malignancy)

                            print(f"Processed {malignant_count + benign_count} files")
                            print(f"Malignancy: {malignancy}")
                            print(f"Patient ID: {pid}")

                        if malignant_count >= max_malignant_count and benign_count >= max_benign_count:
                            break
                    except Exception as e:
                        # Skip problematic nodules that cannot be grouped manually
                        print(f"Skipping nodule processing for Patient ID {pid} due to error: {e}")

                except Exception as e:
                    print(f"Error processing file {filename}: {e}")

        if malignant_count >= max_malignant_count and benign_count >= max_benign_count:
            break

print(f"Total malignant images: {malignant_count}")
print(f"Total benign images: {benign_count}")


Processed 1 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 2 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 3 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 4 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 5 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 6 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 7 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 8 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 9 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 10 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 11 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 12 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 13 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 14 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 15 files
Malignancy: benign
Patient ID: LIDC-IDRI-0107
Processed 16 files
Malignancy: ben

In [4]:
# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

In [5]:
# Normalization
images = images/255.0

In [6]:
# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
labels_encoded = to_categorical(labels_encoded)

# Print the mapping of labels to their encoded values
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
print("Label Mapping:", label_mapping)


Label Mapping: {'benign': 0, 'malignant': 1}


In [7]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels_encoded, test_size=0.2, random_state=42, shuffle=True)

# Convert to numpy arrays and reshape
X_train = np.array(X_train).reshape(-1, 300, 300, 1)
X_test = np.array(X_test).reshape(-1, 300, 300, 1)

# Define number of classes
num_classes = len(np.unique(labels_encoded))

print(num_classes)

2


In [8]:
import tensorflow as tf
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.layers import Input, BatchNormalization, Activation, Conv2D, concatenate, AveragePooling2D, ZeroPadding2D, MaxPooling2D, GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

def conv_block(x, growth_rate, name):
    """A building block for a dense block."""
    bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
    x1 = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(x)
    x1 = Activation('relu', name=name + '_0_relu')(x1)
    x1 = Conv2D(4 * growth_rate, 1, use_bias=False, name=name + '_1_conv')(x1)
    x1 = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(x1)
    x1 = Activation('relu', name=name + '_1_relu')(x1)
    x1 = Conv2D(growth_rate, 3, padding='same', use_bias=False, name=name + '_2_conv')(x1)
    x = concatenate([x, x1], axis=bn_axis, name=name + '_concat')
    return x

def transition_block(x, reduction, name):
    """A transition block."""
    bn_axis = 3 if K.image_data_format() == 'channels_last' else 1
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_bn')(x)
    x = Activation('relu', name=name + '_relu')(x)
    x = Conv2D(int(tf.keras.backend.int_shape(x)[bn_axis] * reduction), 1, use_bias=False, name=name + '_conv')(x)
    x = AveragePooling2D(2, strides=2, name=name + '_pool')(x)
    return x

def dense_block(x, blocks, name):
    """A dense block."""
    for i in range(blocks):
        x = conv_block(x, 32, name=name + '_block' + str(i + 1))
    return x

def DenseNet201(input_shape=None, classes=2):
    """Instantiates the DenseNet-201 architecture."""
    img_input = Input(shape=input_shape)

    bn_axis = 3 if K.image_data_format() == 'channels_last' else 1

    x = ZeroPadding2D(padding=((3, 3), (3, 3)))(img_input)
    x = Conv2D(64, 7, strides=2, use_bias=False, name='conv1_conv')(x)
    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='conv1_bn')(x)
    x = Activation('relu', name='conv1_relu')(x)
    x = ZeroPadding2D(padding=((1, 1), (1, 1)))(x)
    x = MaxPooling2D(3, strides=2, name='pool1')(x)

    x = dense_block(x, 6, name='conv2')
    x = transition_block(x, 0.5, name='pool2')
    x = dense_block(x, 12, name='conv3')
    x = transition_block(x, 0.5, name='pool3')
    x = dense_block(x, 48, name='conv4')
    x = transition_block(x, 0.5, name='pool4')
    x = dense_block(x, 32, name='conv5')

    x = BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name='bn')(x)
    x = Activation('relu', name='relu')(x)

    x = GlobalAveragePooling2D(name='avg_pool')(x)
    x = Dense(classes, activation='softmax', name='fc')(x)

    # Create model.
    model = Model(img_input, x, name='densenet201')

    return model

# Example usage:
model = DenseNet201(input_shape=(300, 300, 1), classes=2)


In [9]:
model.summary()

Model: "densenet201"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d      │ (None, 306, 306,  │          0 │ input_layer[0][0] │
│ (ZeroPadding2D)     │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_conv (Conv2D) │ (None, 150, 150,  │      3,136 │ zero_padding2d[0… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_bn            │ (None, 150, 150,  │        256 │ conv1_conv[0][0]  │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1_relu          │ (None, 150, 150,  │          0 │ conv1_bn[0][0]    │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ zero_padding2d_1    │ (None, 152, 152,  │          0 │ conv1_relu[0][0]  │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pool1               │ (None, 75, 75,    │          0 │ zero_padding2d_1… │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_bn   │ (None, 75, 75,    │        256 │ pool1[0][0]       │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_0_relu │ (None, 75, 75,    │          0 │ conv2_block1_0_b… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_conv │ (None, 75, 75,    │      8,192 │ conv2_block1_0_r… │
│ (Conv2D)            │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_bn   │ (None, 75, 75,    │        512 │ conv2_block1_1_c… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_1_relu │ (None, 75, 75,    │          0 │ conv2_block1_1_b… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_2_conv │ (None, 75, 75,    │     36,864 │ conv2_block1_1_r… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block1_concat │ (None, 75, 75,    │          0 │ pool1[0][0],      │
│ (Concatenate)       │ 96)               │            │ conv2_block1_2_c… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_bn   │ (None, 75, 75,    │        384 │ conv2_block1_con… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_0_relu │ (None, 75, 75,    │          0 │ conv2_block2_0_b… │
│ (Activation)        │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2_block2_1_conv │ (None, 75, 75,    │     12,288 │ conv2_block2_0_r

 Total params: 18,319,554 (69.88 MB)

 Trainable params: 18,090,498 (69.01 MB)

 Non-trainable params: 229,056 (894.75 KB)

In [10]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model

# Assuming the DenseNet201 is defined as above
def get_model():
    return DenseNet201(input_shape=(300, 300, 1), classes=2)

k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

# Placeholder arrays to store performance metrics
accuracy_scores = []

# Perform cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(X_train)):
    print(f'Fold {fold + 1}/{k}')
    
    # Split data into folds
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
    # Reinitialize the model for each fold
    model = get_model()  # Use the function that returns your DenseNet201 model
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

    # Filepath to save the best model of this fold
    checkpoint_filepath = f'/kaggle/working/best_model_fold_{fold + 1}.keras'
    
    # ModelCheckpoint to save the best weights
    checkpoint = ModelCheckpoint(checkpoint_filepath, 
                                 monitor='val_accuracy', 
                                 save_best_only=True, 
                                 mode='max', 
                                 verbose=1)

    # EarlyStopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_accuracy', 
                                   patience=3,  # Stop training if no improvement after 3 epochs
                                   mode='max',
                                   verbose=1,
                                   restore_best_weights=True)
    
    # Train the model with early stopping and checkpoint callbacks
    model.fit(X_train_fold, y_train_fold, 
              epochs=5,  # Increase if needed
              batch_size=32, 
              validation_data=(X_val_fold, y_val_fold),
              verbose=1,
              callbacks=[checkpoint, early_stopping])

    # Load the best model saved during this fold
    model = load_model(checkpoint_filepath)

    # Evaluate on the validation fold
    scores = model.evaluate(X_val_fold, y_val_fold, verbose=0)
    accuracy_scores.append(scores[1])  # Save accuracy from the scores

# Print average accuracy across all folds
avg_accuracy = np.mean(accuracy_scores)
print(f'Average Accuracy across {k}-folds: {avg_accuracy:.4f}')


Fold 1/5
Epoch 1/5


I0000 00:00:1726390538.319222     102 service.cc:145] XLA service 0x7f4d080067b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726390538.319277     102 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1726390538.319282     102 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1726390761.565608     102 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_fusion_92', 8 bytes spill stores, 8 bytes spill loads

I0000 00:00:1726390761.987192     102 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 766ms/step - accuracy: 0.7112 - loss: 0.5774
Epoch 1: val_accuracy improved from -inf to 0.34464, saving model to /kaggle/working/best_model_fold_1.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 449s 954ms/step - accuracy: 0.7119 - loss: 0.5764 - val_accuracy: 0.3446 - val_loss: 1.5176
Epoch 2/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 765ms/step - accuracy: 0.9522 - loss: 0.1328
Epoch 2: val_accuracy improved from 0.34464 to 0.43036, saving model to /kaggle/working/best_model_fold_1.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 141s 834ms/step - accuracy: 0.9523 - loss: 0.1327 - val_accuracy: 0.4304 - val_loss: 1.4702
Epoch 3/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 761ms/step - accuracy: 0.9853 - loss: 0.0475
Epoch 3: val_accuracy improved from 0.43036 to 0.87946, saving model to /kaggle/working/best_model_fold_1.keras
140/140 ━━━━━━━━━━━━━━━━━━━━ 116s 831ms/step - accuracy: 0.9853 - loss: 0.0474 - val_accuracy: 0.8795 - val_loss: 0.5935
Epoch 4/5
140/140 ━━━━━━━━━━━━━━━━━━━━ 0s 758ms/step 